## Import

In [6]:
# set the environment path to find Recommenders
import sys
import itertools
import logging
import os
import xlwings as xw
import numpy as np
import pandas as pd
import papermill as pm

from recommenders.datasets import movielens
from recommenders.datasets.python_splitters import python_stratified_split
from recommenders.evaluation.python_evaluation import map_at_k, ndcg_at_k, precision_at_k, recall_at_k, logloss
from recommenders.models.sar import SAR

print("System version: {}".format(sys.version))
print("Pandas version: {}".format(pd.__version__))

System version: 3.9.13 (main, Aug 25 2022, 23:51:50) [MSC v.1916 64 bit (AMD64)]
Pandas version: 1.4.4


In [2]:
# top k items to recommend
TOP_K = 10

## Load data

In [16]:
def read_xlsx(name, sheet_cnt):
    for idx in range(sheet_cnt):
        instance = xw.App(visible = False)
        xlsx_data = xw.Book(name).sheets[idx]
        
        if idx == 0:
            output_df = xlsx_data.range('A1').options(pd.DataFrame, index = False, expand = 'table').value
        else:
            temp_df = xlsx_data.range('A1').options(pd.DataFrame, index = False, expand = 'table').value
            output_df = pd.concat([output_df, temp_df])
    instance.quit()
    instance.kill()
    
    return output_df

def read_data(name):
    try :
        output = pd.read_excel(name)
    except ValueError:    
        output = read_xlsx(name)
    return output

In [43]:
data_path = os.getcwd() + "\\datasets\\"

In [44]:
reco = read_xlsx(data_path + 'recommender_base_v2.xlsx', 2)

In [45]:
reco.head()

,FAKE_CANO,PRDT_TYPE_CD,CATEGORY,CNT,TR_DT,TARGET
0,1.0,152,랩어카운트,4.0,20180711,TRAIN
1,1.0,300,주식,36.0,20210323,TRAIN
2,1.0,325,발행어음,4.0,20190918,TRAIN
3,1.0,306,ELS,6.0,20200929,TRAIN
4,1.0,307,펀드,20.0,20200506,TRAIN


* find train/test cano

In [46]:
temp_df = pd.DataFrame(reco.groupby(['FAKE_CANO', 'TARGET'])['PRDT_TYPE_CD'].count()).reset_index()
temp_df2 = pd.DataFrame(temp_df.groupby("FAKE_CANO")['TARGET'].count()).reset_index()
split_cano = temp_df2.loc[temp_df2.TARGET > 1, ['FAKE_CANO']]

In [47]:
reco = reco.loc[reco.FAKE_CANO.isin(split_cano.FAKE_CANO), :]

* data split

In [48]:
reco.head()

,FAKE_CANO,PRDT_TYPE_CD,CATEGORY,CNT,TR_DT,TARGET
6,4.0,152,랩어카운트,21.0,20220222,TEST
7,4.0,300,주식,18.0,20220105,TEST
8,4.0,302,채권,30.0,20220114,TEST
9,4.0,307,펀드,240.0,20220103,TEST
10,4.0,325,발행어음,87.0,20220103,TEST


In [49]:
train_reco = reco.loc[reco.TARGET == 'TRAIN']
test_reco = reco.loc[reco.TARGET == 'TEST']

In [54]:
train_reco.loc[train_reco.FAKE_CANO == 4]

,FAKE_CANO,PRDT_TYPE_CD,CATEGORY,CNT,TR_DT,TARGET
11,4.0,325,발행어음,87.0,20211214,TRAIN
12,4.0,300,주식,69.0,20211118,TRAIN
13,4.0,302,채권,168.0,20211108,TRAIN
14,4.0,306,ELS,141.0,20210917,TRAIN
15,4.0,307,펀드,3060.0,20211213,TRAIN
16,4.0,318,단기사채,30.0,20190722,TRAIN
17,4.0,152,랩어카운트,75.0,20211130,TRAIN


In [56]:
test_reco.loc[test_reco.FAKE_CANO == 4]

,FAKE_CANO,PRDT_TYPE_CD,CATEGORY,CNT,TR_DT,TARGET
6,4.0,152,랩어카운트,21.0,20220222,TEST
7,4.0,300,주식,18.0,20220105,TEST
8,4.0,302,채권,30.0,20220114,TEST
9,4.0,307,펀드,240.0,20220103,TEST
10,4.0,325,발행어음,87.0,20220103,TEST


* get rating

In [57]:
def portion(x):
    return x / x.sum()

In [59]:
def base_preproc(reco):
    sum_df = pd.DataFrame(reco.groupby(['FAKE_CANO'])['CNT'].sum()).reset_index()
    sum_df.columns = ['FAKE_CANO', 'TOT']
    reco = pd.merge(reco, sum_df, on = 'FAKE_CANO', how = 'inner')
    reco['Rating'] = reco['CNT'] / reco['TOT']
    reco['Rating'] = reco['Rating'].astype(np.float32)
    
    return reco

In [61]:
train_reco = base_preproc(train_reco)
test_reco = base_preproc(test_reco)

In [62]:
train_reco.head()

,FAKE_CANO,PRDT_TYPE_CD,CATEGORY,CNT,TR_DT,TARGET,TOT,Rating
0,4.0,325,발행어음,87.0,20211214,TRAIN,3630.0,0.023967
1,4.0,300,주식,69.0,20211118,TRAIN,3630.0,0.019008
2,4.0,302,채권,168.0,20211108,TRAIN,3630.0,0.046281
3,4.0,306,ELS,141.0,20210917,TRAIN,3630.0,0.038843
4,4.0,307,펀드,3060.0,20211213,TRAIN,3630.0,0.842975


In [63]:
header = {
    "col_user": "FAKE_CANO",
    "col_item": "CATEGORY",
    "col_rating": "Rating",
#    "col_timestamp": "Timestamp",
#    "col_prediction": "Prediction",
    "col_timestamp": None,
    "col_prediction": "Prediction",
}

In [12]:
# train, test = python_stratified_split(reco, ratio=0.75, col_user=header["col_user"], col_item=header["col_item"], seed=42)

In this case, for the illustration purpose, the following parameter values are used:

|Parameter|Value|Description|
|---------|---------|-------------|
|`similarity_type`|`jaccard`|Method used to calculate item similarity.|
|`time_decay_coefficient`|30|Period in days (term of $T$ shown in the formula of Section 1.2)|
|`time_now`|`None`|Time decay reference.|
|`timedecay_formula`|`True`|Whether time decay formula is used.|

In [64]:
# set log level to INFO
logging.basicConfig(level=logging.DEBUG, 
                    format='%(asctime)s %(levelname)-8s %(message)s')

model = SAR(
    similarity_type="jaccard", 
#    time_decay_coefficient=30, 
#    time_now=None, 
#    timedecay_formula=True, 
    **header
)

In [65]:
model.fit(train_reco)

2022-12-08 13:55:09,083 INFO     Collecting user affinity matrix
2022-12-08 13:55:09,132 INFO     Creating index columns
2022-12-08 13:55:10,876 INFO     Building user affinity sparse matrix
2022-12-08 13:55:10,909 INFO     Calculating item co-occurrence
2022-12-08 13:55:11,012 INFO     Calculating item similarity
2022-12-08 13:55:11,013 INFO     Using jaccard based similarity
2022-12-08 13:55:11,015 INFO     Done training


In [67]:
TOP_K = 5
top_k = model.recommend_k_items(test_reco, top_k=TOP_K, remove_seen=False)

2022-12-08 13:55:20,100 INFO     Calculating recommendation scores


In [68]:
top_k

,FAKE_CANO,CATEGORY,Prediction
0,4.0,펀드,0.884193
1,4.0,채권,0.359644
2,4.0,ELS,0.345251
3,4.0,RP,0.279993
4,4.0,발행어음,0.234176
...,...,...,...
459175,3705257.0,주식,0.971079
459176,3705257.0,해외주식-NASD,0.936858
459177,3705257.0,해외주식-NYSE,0.917172
459178,3705257.0,해외주식-AMEX,0.885262


In [74]:
test_reco

,FAKE_CANO,PRDT_TYPE_CD,CATEGORY,CNT,TR_DT,TARGET,TOT,Rating
0,4.0,152,랩어카운트,21.0,20220222,TEST,396.0,0.053030
1,4.0,300,주식,18.0,20220105,TEST,396.0,0.045455
2,4.0,302,채권,30.0,20220114,TEST,396.0,0.075758
3,4.0,307,펀드,240.0,20220103,TEST,396.0,0.606061
4,4.0,325,발행어음,87.0,20220103,TEST,396.0,0.219697
...,...,...,...,...,...,...,...,...
394418,3694273.0,529,해외주식-AMEX,2.0,20220107,TEST,26.0,0.076923
394419,3705257.0,513,해외주식-NYSE,8.0,20220209,TEST,42.0,0.190476
394420,3705257.0,529,해외주식-AMEX,6.0,20220107,TEST,42.0,0.142857
394421,3705257.0,300,주식,12.0,20220308,TEST,42.0,0.285714


* evaluate model performance

In [69]:
header

{'col_user': 'FAKE_CANO',
 'col_item': 'CATEGORY',
 'col_rating': 'Rating',
 'col_timestamp': None,
 'col_prediction': 'Prediction'}

In [71]:
# all ranking metrics have the same arguments
args = [test_reco, top_k]
kwargs = dict(col_user=header['col_user'], 
              col_item=header['col_item'], 
              col_rating=header['col_rating'] ,
              col_prediction=header['col_prediction'] ,
              relevancy_method='top_k', 
              k=TOP_K)

eval_map = map_at_k(*args, **kwargs)
eval_ndcg = ndcg_at_k(*args, **kwargs)
eval_precision = precision_at_k(*args, **kwargs)
eval_recall = recall_at_k(*args, **kwargs)

In [72]:
print(f"Model:",
      f"Top K:\t\t {TOP_K}",
      f"MAP:\t\t {eval_map:f}",
      f"NDCG:\t\t {eval_ndcg:f}",
      f"Precision@K:\t {eval_precision:f}",
      f"Recall@K:\t {eval_recall:f}", sep='\n')

Model:
Top K:		 5
MAP:		 0.891466
NDCG:		 0.933454
Precision@K:	 0.764670
Recall@K:	 0.907229


* check log_loss

In [19]:
test_log = test
test_log['Rating'] = 1

args_log = [test_log, top_k]
kwargs_log = dict(col_user=header['col_user'], 
              col_item=header['col_item'], 
              col_rating=header['col_rating'] ,
              col_prediction=header['col_prediction'])

In [ ]:
logloss(*args_log, **kwargs_log)